# Librerías

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
import torch.nn.functional as F
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import sys
import json
import os
import random
import time

from urllib.request import Request, urlopen, urlretrieve
from typing import List
from pprint import pprint
from PIL import Image
from typing_extensions import Required
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.utils.data.dataset import random_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM

# Extracción de datos con web scrapping


In [2]:
# son 905, pero los últimos no tienen data
max_pokemon: int = 898

In [3]:
types = ["grass", "fire", "water", "electric"]

In [4]:
dir1: str = "official-artwork"
if os.path.exists(dir1) == False:
    os.mkdir(dir1)

In [5]:
dir2: str = "all"
if os.path.exists(dir2) == False:
    os.mkdir(dir2)

In [ ]:
""" 
SOLO PARA DESCARGAR DATOS
"""
# for pokemon_id in range(max_pokemon):
#     req: Request = Request("https://pokeapi.co/api/v2/pokemon/"+str(pokemon_id+1), 
#                     headers={'User-Agent': 'Mozilla/5.0'})
#     webpage: bytes = urlopen(req).read()
#     json_obj: dict = json.loads(webpage)
#     img_url: str = json_obj["sprites"]["other"]["official-artwork"]["front_default"]
    
#     name: str = json_obj["species"]["name"]
#     type: str = json_obj["types"][0]["type"]["name"]

#     if os.path.exists(dir1+"/"+type) == False:
#         os.mkdir(dir1+"/"+type)
    
#     img: str = img_url.split("/")[-1]
#     urlretrieve(img_url, dir1+"/"+type+"/"+name+".png")

In [6]:
imgs_category: list = ["official-artwork", "home", "dream_world"]

In [7]:
for type in types:
    if os.path.exists(dir2+"/"+type) == False:
        os.mkdir(dir2+"/"+type)
    # if os.path.exists("svg/"+type) == False:
    #     os.mkdir("svg/"+type)

In [ ]:
""" 
SOLO PARA DESCARGAR DATOS
"""
# for pokemon_id in range(max_pokemon):
#     print(pokemon_id+1)
#     req: Request = Request("https://pokeapi.co/api/v2/pokemon/"+str(pokemon_id+1), 
#                     headers={'User-Agent': 'Mozilla/5.0'})
#     webpage: bytes = urlopen(req).read()
#     json_obj: dict = json.loads(webpage)
#     type: str = json_obj["types"][0]["type"]["name"]
#     if type not in types:
#         continue
#     name: str = json_obj["species"]["name"]

#     for img_category in imgs_category:
#         img_url: str = json_obj["sprites"]["other"][img_category]["front_default"]
#         if img_url == None:
#             continue
#         save_route: str = "/"+type+"/"+name+"_"+img_category+"."
#         format: str = img_url[-3:]
#         if format == "svg":
#             urlretrieve(img_url, "svg"+save_route+format)
#             drawing = svg2rlg("svg"+save_route+format)
#             renderPM.drawToFile(drawing, dir2+save_route+"png", fmt="PNG")
#         else:
#             urlretrieve(img_url, dir2+save_route+format)

In [8]:
# Distribución de los datos
for type in types:
    print(type, len(os.listdir(dir1+"/"+type)))
# types.remove("flying")
print(types)


grass 86
fire 58
water 123
electric 49
['grass', 'fire', 'water', 'electric']


In [9]:
# Distribución de los datos
sum: int = 0
for type in types:
    freq: int = len(os.listdir(dir2+"/"+type))
    sum += freq
    print(type, freq)
# types.remove("flying")
print(sum)

grass 233
fire 155
water 346
electric 131
865


# CNN

## Alexnet

In [ ]:
model = torchvision.models.alexnet(pretrained=True)
for e in model.parameters():
   e.requires_grad = False
model.classifier[4] = nn.Linear(4096, 1024)
model.classifier[6] = nn.Linear(1024, len(types))
# model.classifier[6] = nn.Linear(4096, len(types))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(model)

In [48]:
x: List[str] = []
y: List[str] = []
for type in types:
    pokemon_imgs: List[str] = os.listdir(dir2+"/"+type)
    for pokemon_img in pokemon_imgs:
        x.append(dir2+"/"+type+"/"+pokemon_img)
        y.append(type)

In [49]:
class MyDataset(Dataset):
      def __init__(self, list_IDs, labels):
            self.labels = labels
            self.list_IDs = list_IDs

      def __len__(self):
            return len(self.list_IDs)

      def __getitem__(self, index):
            # Select sample
            ID = self.list_IDs[index]

            # Load data and get label
            input_image = Image.open(ID)
            input_image = input_image.convert('RGB')
            preprocess = transforms.Compose([
                  transforms.Resize(256),
                  transforms.CenterCrop(224),
                  transforms.RandomHorizontalFlip(),
                  transforms.ToTensor()
            ])
            X = preprocess(input_image)
            y = self.labels[index]
            return X, y

In [39]:
le = preprocessing.LabelEncoder()
tensor_y = le.fit_transform(y)
tensor_y = torch.as_tensor(tensor_y)
dataset = MyDataset(x, tensor_y)


In [50]:
data_train, data_val, data_test = random_split(dataset=dataset, lengths=[605, 87, 173])

In [51]:
batch_size = 64
train_loader = DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=data_val, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=data_test, batch_size=batch_size, shuffle=False)

In [52]:
loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
optimizer = optim.SGD(model.parameters(), lr=3e-4, weight_decay=0.005, momentum=0.9)
# optimizer = optim.SGD(model.parameters(), lr=0.005)
total_step = len(train_loader)
num_epochs = 100

In [27]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [44]:
def train(model, optimizer, loss_fn, num_epochs) -> List[float]:
  # train the model
  total_step: int = len(train_loader)
  print(total_step)

  list_train_loss: List[float] = []
  list_val_loss: List[float] = []

  beg = time.time()
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)
      # forward 
      output = model(images)
      loss   = loss_fn(output, labels)
      # change the params
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      # del images, labels, output
    
    train_loss: float = loss.item()
    list_train_loss.append(train_loss)
    with torch.no_grad():
        model.eval()
        for images, labels in val_loader:
          correct = 0
          total = 0
          for images, labels in val_loader:
              images = images.to(device)
              labels = labels.to(device)
              outputs = model(images)
              loss = loss_fn(outputs, labels)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
              del images, labels, outputs

        val_loss: float = loss.detach().cpu()
        list_val_loss.append(val_loss)
        print ('Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss {:.4f},  Accuracy: {:.4f}'
              .format(epoch+1, num_epochs, train_loss, val_loss, 100*correct/total))
  end = time.time()
  print('Finished training trainset in {} seconds'.format(round(end-beg, 2)))

  return list_train_loss, list_val_loss

In [45]:
def test(model, save_name: str = ""):
    with torch.no_grad():
        all_predicted: list = []
        all_labels: list = []
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_predicted.extend((predicted).tolist())
            all_labels.extend((labels).tolist())
        val = (np.unique(le.inverse_transform(all_predicted)))
        matrix = confusion_matrix(all_labels, all_predicted)
        df = pd.DataFrame(matrix, index=val, columns=val)
        sns.heatmap(df, annot=True, cbar=None, cmap="Greens")
        plt.title("Confusion Matrix"), plt.tight_layout()
        plt.xlabel("Predicted")
        plt.ylabel("Real")
        if save_name == "":
            plt.show()
        else:
            plt.savefig(PATH_RES+save_name)
        print('Test of the model on the {} test images'.format(len(data_test)))
        print('Acurracy: {:.2f} %'.format(100*accuracy_score(all_labels, all_predicted)))
        print('F1-score: {:.2f} %'.format(100*f1_score(all_labels, all_predicted, average='macro')))
        print('Recall: {:.2f} %'.format(100*recall_score(all_labels, all_predicted, average='macro')))
        print('Precision: {:.2f} %'.format(100*precision_score(all_labels, all_predicted, average='macro')))


In [ ]:
# test before train
test(model)

In [54]:
train_loss, val_loss = train(model, optimizer, loss_fn, 100)

10
Epoch [1/100], Train Loss: 1.1672, Validation Loss 1.1509,  Accuracy: 52.8736
Epoch [2/100], Train Loss: 0.8134, Validation Loss 0.9039,  Accuracy: 60.9195
Epoch [3/100], Train Loss: 0.8545, Validation Loss 0.7771,  Accuracy: 70.1149
Epoch [4/100], Train Loss: 0.6088, Validation Loss 0.7959,  Accuracy: 66.6667
Epoch [5/100], Train Loss: 0.6903, Validation Loss 0.6928,  Accuracy: 68.9655
Epoch [6/100], Train Loss: 0.5418, Validation Loss 0.6854,  Accuracy: 72.4138
Epoch [7/100], Train Loss: 0.5656, Validation Loss 0.6352,  Accuracy: 72.4138
Epoch [8/100], Train Loss: 0.7385, Validation Loss 0.6013,  Accuracy: 72.4138
Epoch [9/100], Train Loss: 0.5759, Validation Loss 0.6075,  Accuracy: 71.2644
Epoch [10/100], Train Loss: 0.4402, Validation Loss 0.5731,  Accuracy: 73.5632
Epoch [11/100], Train Loss: 0.4763, Validation Loss 0.6028,  Accuracy: 73.5632
Epoch [12/100], Train Loss: 0.5027, Validation Loss 0.5582,  Accuracy: 72.4138
Epoch [13/100], Train Loss: 0.5463, Validation Loss 0.5074

In [55]:
PATH_RES: str = "results/"

In [ ]:
# test after train
torch.manual_seed(1)
test(model, "alexnet confusion matrix SGD epochs=100 lr=3e-4 weight_decay=0.005 momentum=0.9.png")
# test(model, "alexnet confusion matrix Adam epochs=100 lr=0.01.png")
# test(model, "alexnet confusion (sin modificar classifier alexnet) matrix SGD epochs=100 lr=3e-4 weight_decay=0.005 momentum=0.9.png")

In [ ]:
plt.plot(train_loss, label="train loss")
plt.plot(val_loss, label="val loss")
plt.ylabel("loss")
plt.xlabel("num epochs")
plt.title("Loss vs num epochs")
plt.legend()
plt.savefig(PATH_RES+"alexnet loss SGD epochs=100 lr=3e-4 weight_decay=0.005 momentum=0.9.png")
# plt.savefig(PATH_RES+"alexnet loss Adam epochs=100 lr=0.01.png")
# plt.savefig(PATH_RES+"alexnet loss (sin modificar classifier alexnet) SGD epochs=100 lr=3e-4 weight_decay=0.005 momentum=0.9.png")
# plt.show()

# Resnet

In [ ]:
resnet18 = torchvision.models.resnet18(pretrained=True)
for e in resnet18.parameters():
   e.requires_grad = False
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet18.fc = nn.Linear(in_features=512, out_features=len(types), bias=True)
optimizer = optim.SGD(resnet18.parameters(), lr=3e-4, weight_decay=0.005, momentum=0.9)
resnet18.to(device)

In [ ]:
train_loss_resnet18, val_loss_resnet18 = train(resnet18, optimizer, loss_fn, 100)

In [ ]:
plt.plot(train_loss_resnet18, label="train loss")
plt.plot(val_loss_resnet18, label="val loss")
plt.ylabel("loss")
plt.xlabel("num epochs")
plt.title("Loss vs num epochs")
plt.legend()
# plt.show()
plt.savefig(PATH_RES+"resnet18 loss SGD epochs=100 lr=3e-4 weight_decay=0.005 momentum=0.9.png")

In [ ]:
torch.manual_seed(1)
test(resnet18, "resnet18 confusion matrix SGD epochs=100 lr=3e-4 weight_decay=0.005 momentum=0.9.png")